In [1]:

points_notin_water = water_points[~water_points['isin_water']].copy()
#print("Points not in water geometry type:", points_notin_water.geometry.type.unique())
water_union = water_shp.unary_union

# Function to find nearest point on water body
def get_nearest_water_point(point, water_geo):
    nearest_geo = nearest_points(point, water_geo)[1]
    distance = point.distance(nearest_geo)
    return nearest_geo, distance


nearest_coords = []
distances = []
for idx, row in points_notin_water.iterrows():
    point_geo = row.geometry
    nearest_point, distance = get_nearest_water_point(point_geo, water_union)
    nearest_coords.append((nearest_point.x, nearest_point.y))
    distances.append(distance)

# Add the results to the GeoDataFrame
points_notin_water['nearest_coord'] = nearest_coords
points_notin_water['distance'] = distances

columns_to_keep = ['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va', 
                   'geometry', 'nearest_coord', 'distance', 'isin_water']
points_notin_water = points_notin_water[columns_to_keep].copy()
print(points_notin_water.head())

water_shp = gpd.read_file('../lake_shape/updatedValidLakes.shp')
water_shp = water_shp.to_crs(epsg=4326)

sites_df = pd.read_csv('data/siteno_matcher.csv') 
sites_geo = [Point(xy) for xy in zip(sites_df['dec_long_va'], sites_df['dec_lat_va'])]
sites_gdf = gpd.GeoDataFrame(sites_df, geometry=sites_geo)
sites_gdf.set_crs(epsg=4326, inplace=True)

print(water_shp.crs, sites_gdf.crs)

water_points = gpd.sjoin(sites_gdf, water_shp, how='left', predicate='within')
water_points['isin_water'] = water_points['index_right'].notnull()
#print(water_points[['site_no', 'is_in_water']])
#water_points.tocsv('data/water_points.csv')

import folium

# Initialize the map at a central location
m = folium.Map(location=[37.5, -122], zoom_start=9, tiles='cartodbpositron')

# Add points to the map
for idx, row in points_notin_water.iterrows():
    folium.CircleMarker(
        location=[row['dec_lat_va'], row['dec_long_va']],
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        popup=f"Site: {row['site_no']}, Dist: {row['distance']:.2f} km"
    ).add_to(m)

    # Add lines to the nearest water point
    folium.PolyLine(
        locations=[
            [row['dec_lat_va'], row['dec_long_va']],
            [row['nearest_coord'][1], row['nearest_coord'][0]]  # Assuming these are latitude and longitude
        ],
        color='blue'
    ).add_to(m)

# Show the map
m
m.save('site_map.html')

NameError: name 'water_points' is not defined